In [ ]:
import sys
sys.path.append("/home/azhuavlev/PycharmProjects/ml-neuman_mano/pytorch3d_nerf")

%load_ext autoreload
%autoreload 2

import lightning as L
from lightning.pytorch.loggers import TensorBoardLogger
from torch.utils.data import DataLoader

import glob
import os
import glob
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pytorch3d
import sys
import time
import torch
from IPython import display
from PIL import Image
from pytorch3d.renderer import (
    FoVPerspectiveCameras,
    NDCMultinomialRaysampler,
    MonteCarloRaysampler,
    EmissionAbsorptionRaymarcher,
    ImplicitRenderer,
    RayBundle,
    ray_bundle_to_ray_points,
)
# Data structures and functions for rendering
from pytorch3d.structures import Volumes
from pytorch3d.transforms import so3_exp_map
from tqdm import tqdm

from datasets import dataset_extr_to_mano, dataset_canonical_space
import lighning_models

from nerf_models import nerf_big_no_warp

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# disable pytorch gradient computation
torch.set_grad_enabled(False)

data_path = '/home/azhuavlev/Desktop/Data/InterHand_Neuman/03'
all_ids = list(range(len(glob.glob(os.path.join(data_path, 'cameras', '*.json')))))

# use 80% of the data for training, randomize the order
np.random.shuffle(all_ids)
train_ids = all_ids[:int(0.6 * len(all_ids))]
test_ids = all_ids[int(0.6 * len(all_ids)):]
print(test_ids)

train_dataset = dataset_extr_to_mano.NeumanDataset(data_path, train_ids)
test_dataset = dataset_extr_to_mano.NeumanDataset(data_path, test_ids)
full_dataset = dataset_extr_to_mano.NeumanDataset(data_path, all_ids)

# We sample 6 random cameras in a minibatch.
batch_size = 1

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=5)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
full_loader = DataLoader(full_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
field = nerf_big_no_warp.NeuralRadianceField()

# ckpt_path = '/itet-stor/azhuavlev/net_scratch/Projects/Results/neuman_custom/lightning_logs/big_sil_loss_1000_mask_0.05_dilation_10_sampling_4096_64_depth_105_huber_mask_huber/checkpoints/epoch=1999-step=42000.ckpt'
# ckpt_path = '/itet-stor/azhuavlev/net_scratch/Projects/Results/neuman_custom/lightning_logs/big_clipped_sil_loss_99999_lr_99999_mask_1_dilation_10_sampling_8192_32_depth_105_huber_mask_huber/checkpoints/epoch=9999-step=210000.ckpt'
# ckpt_path = '/home/azhuavlev/Desktop/Results/neuman_custom/lightning_logs/small_warp_clipped_sil_loss_99999_lr_99999_mask_0.3_dilation_10_sampling_8192_32_depth_105_huber/checkpoints/epoch=2199-step=79200.ckpt'
ckpt_path = '/home/azhuavlev/Desktop/Results/neuman_custom/lightning_logs/big_allLosses/checkpoints/epoch=5999-step=126000.ckpt'

model = lighning_models.HandModel(nerf_model=field).load_from_checkpoint(ckpt_path,
                                                                                             dataset=full_loader,
                                                                                             nerf_model=field)
model = model.to('cuda')

In [ ]:
# generate points in 3d space between -0.3 and 0.3
with torch.no_grad():
    point_cloud = torch.linspace(-0.1, 0.2, 400, device='cpu')
    points = torch.stack(torch.meshgrid(point_cloud, point_cloud, point_cloud), dim=-1).reshape(1, 1, -1, 1, 3)
    # directions = torch.zeros(1, 1, points.shape[2], 3, device = points.device)
    directions = torch.ones_like(points)
print('points', points.shape)
print('directions', directions.shape)

In [ ]:
from tqdm import tqdm
# iterate over points in chunks of 1000000:
rays_densities = torch.tensor([], device='cpu')
rays_colors = torch.tensor([], device='cpu')

with torch.no_grad():
    for i in tqdm(range(0, points.shape[2], 1000000)):
        points_batch = points[:, :, i:i+1000000,:,  :].to('cuda')
        directions_batch = directions[:, :, i:i+1000000, :].to('cuda')
        rays_densities_batch, rays_colors_batch = model.get_nerf_output_canonical(points_batch, directions_batch)

        rays_densities = torch.cat((rays_densities, rays_densities_batch.cpu()), dim=2)
        rays_colors = torch.cat((rays_colors, rays_colors_batch.cpu()), dim=2)

In [ ]:
densities_numpy = rays_densities.cpu().numpy()
for i in [0.01, 0.1, 0.3, 0.5, 0.9]:
    print(f'fraction occupied > {i:.2f} - {np.mean(densities_numpy > i) * 100:.1f}%')

In [ ]:
import trimesh

points_reshaped = points.reshape(-1, 3)
rays_colors_reshaped = rays_colors.reshape(-1, 3)
rays_densities_reshaped = rays_densities.reshape(-1, 1)

threshold = 0.01

points_reshaped = points_reshaped[rays_densities.reshape(-1) > threshold].cpu().numpy()
rays_colors_reshaped = rays_colors_reshaped[rays_densities.reshape(-1) > threshold].cpu().numpy()
rays_densities_reshaped = rays_densities_reshaped[rays_densities.reshape(-1) > threshold].cpu().numpy()

# print('rays_densities_reshaped.shape', rays_densities_reshaped.shape, rays_densities_reshaped.dtype, rays_densities_reshaped.min(), rays_densities_reshaped.max())
# print('rays_colors_reshaped.shape', rays_colors_reshaped.shape, rays_colors_reshaped.dtype, rays_colors_reshaped.min(), rays_colors_reshaped.max())
rays_colors_rgba = np.concatenate((rays_colors_reshaped, rays_densities_reshaped), axis=1)

# convert colors from [0, 1] to [0, 255]
rays_colors_rgba = rays_colors_rgba * 255
rays_colors_rgba = rays_colors_rgba.astype(np.uint8)

point_cloud = trimesh.PointCloud(points_reshaped, colors=rays_colors_rgba)

scene = trimesh.Scene()
scene.add_geometry(point_cloud)
# show coordinate axes
scene.add_geometry(trimesh.creation.axis(axis_length=0.15, axis_radius = 0.001, origin_size=0.001))

scene.show()

In [ ]:
from skimage import measure
from skimage.draw import ellipsoid
import trimesh

rays_densities_xyz = rays_densities.reshape(400, 400, 400)
verts, faces, normals, values = measure.marching_cubes(rays_densities_xyz.cpu().numpy(), 0.01)
mesh = trimesh.Trimesh(vertices=verts, faces=faces)

# display mesh
mesh.show()